In [1]:
import time

import pandas as pd
import numpy as np

import tomotopy as tp
import pyLDAvis

from konlpy.tag import Okt
from konlpy.utils import pprint
from multiprocessing import Pool

import matplotlib.pyplot as plt
import matplotlib.colors as clr
%matplotlib inline 

C:\Users\LEE\anaconda3\lib\site-packages\scipy\special\orthogonal.py:79: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
C:\Users\LEE\anaconda3\lib\site-packages\scipy\special\orthogonal.py:79: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to re

In [2]:
df = pd.read_csv("최종.csv")
for i in range(0,len(df['Content'])):
    df['Content'][i] = df['Content'][i].lstrip()
    df['Content'][i] = df['Content'][i].rstrip()

<ipython-input-2-105e35590030>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Content'][i] = df['Content'][i].lstrip()
<ipython-input-2-105e35590030>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Content'][i] = df['Content'][i].rstrip()


In [3]:
okt = Okt()
tags = ['Noun']
stopwords = []

def Tokenizer(text):
    morphs = okt.pos(text)
    pos = []

    for word, tag in morphs:
        if(len(word) < 2 and word not in stopwords): 
            continue
        if tag in tags:
            pos.append(word)

    return pos

In [4]:
Tokenizer('안녕하세요 제 이름은 이철 입니다. 이번 수업을 열심히 듣고 있어요')

['이름', '이철', '이번', '수업']

In [6]:
starttime = time.time()

mdl = tp.LDAModel(k=10, min_cf=500, rm_top=10)
# k토픽갯수 min_cf 최소단어언급횟수, rm_top 제거상위단어10개
# mdl = tp.HDPModel(initial_k=10) <- 모델바꾸기

x = len(df['Content'])
for i in range(0,len(df['Content'])):  # text를 doc에 넣어줌
    text = df['Content'][i]
    mdl.add_doc(Tokenizer(text))
    if i % 500 == 0:
        runtime = time.time() - starttime
        print(f'{i}번째 기사 완료, {x-i}개 남음, 현재까지 {runtime}초걸림")')

for i in range(0, 100, 10): # 범위 0부터 100까지 10단위로
    mdl.train(iter=10, workers=0) #반복 10번 워커 다써라
    print(f'Iteration: {i}\tLog-likelihood: {mdl.ll_per_word}')

for k in range(mdl.k):
    print(f'Top 10 words of topic #{k}')
    print(mdl.get_topic_words(k, top_n=10))

runtime = time.time() - starttime
print(f"끝, {runtime}초걸림")



0번째 기사 완료, 14105개 남음, 현재까지 0.03795766830444336초걸림")
500번째 기사 완료, 13605개 남음, 현재까지 18.43073868751526초걸림")
1000번째 기사 완료, 13105개 남음, 현재까지 39.74884533882141초걸림")
1500번째 기사 완료, 12605개 남음, 현재까지 59.922858238220215초걸림")
2000번째 기사 완료, 12105개 남음, 현재까지 82.34525442123413초걸림")
2500번째 기사 완료, 11605개 남음, 현재까지 105.22161197662354초걸림")
3000번째 기사 완료, 11105개 남음, 현재까지 127.48133563995361초걸림")
3500번째 기사 완료, 10605개 남음, 현재까지 150.8490538597107초걸림")
4000번째 기사 완료, 10105개 남음, 현재까지 173.26845026016235초걸림")
4500번째 기사 완료, 9605개 남음, 현재까지 196.05784511566162초걸림")
5000번째 기사 완료, 9105개 남음, 현재까지 216.07198667526245초걸림")
5500번째 기사 완료, 8605개 남음, 현재까지 234.39844703674316초걸림")
6000번째 기사 완료, 8105개 남음, 현재까지 252.4362769126892초걸림")
6500번째 기사 완료, 7605개 남음, 현재까지 271.2390716075897초걸림")
7000번째 기사 완료, 7105개 남음, 현재까지 291.02621603012085초걸림")
7500번째 기사 완료, 6605개 남음, 현재까지 308.8226971626282초걸림")
8000번째 기사 완료, 6105개 남음, 현재까지 326.2908639907837초걸림")
8500번째 기사 완료, 5605개 남음, 현재까지 343.80109095573425초걸림")
9000번째 기사 완료, 5105개 남음, 현재까지 363.2351186275482초걸

In [8]:
topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq
prepared_data = pyLDAvis.prepare(topic_term_dists,doc_topic_dists,doc_lengths,vocab,term_frequency,mds='mmds')


C:\Users\LEE\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\LEE\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\LEE\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py

In [1]:
#pyLDAvis.display(prepared_data)